In [35]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss
import plotly
import math
from statistics import median, mean
from selfpkg import readDB, MinToMS
from dataFilter import butter_lowpass_filter, butter_highpass_filter

In [7]:
timeConversionVal = 60000

In [8]:
#function for calculate the time interval between peaks
def velocityCal(df, peaksIndex):
    dfTime = df.iloc[peaksIndex] #get sub dataframe
    prev = dfTime[0: 1] #cache the first 
    for i, _ in dfTime.iteritems():
        cur = dfTime[i] #cache cur
        dfTime[i] = dfTime[i] - prev #get difference
        prev = cur #set prev from cur (cur val we've cached before)
    return dfTime

In [9]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [10]:
def jointDetectionByCompass(df, filename='jointDetectionByCompass'):
    x = df.compassX
    y = df.compassY
    z = df.compassZ

    v = ((x ** 2 + y ** 2 + z ** 2) ** (1/2))

    ma = movingaverage(v, 500) #get moving average of 500 points, all point > avg, assign avg
    for i in range(len(v)):
        if v[i] > ma[i]:
            v[i] = ma[i]

    v_processed = butter_highpass_filter(butter_lowpass_filter(v, 2.5, 50, 5), 0.5, 50, 5) # lowpass + high pass

    for i in range(len(v_processed)): # all point < 1, assign 0
        if v_processed[i] < 1:
            v_processed[i] = 0

    i = 0
    j = 0
    width = 25 #ger rid of peak whose width > 25 points
    while(j < len(v_processed)):
        if v_processed[i] == 0 and v_processed[j] == 0:
            i = i + 1
            j = j + 1
        elif v_processed[i] != 0 and v_processed[j] != 0:
            j = j + 1
        elif v_processed[i] != 0 and v_processed[j] == 0:
            if j - i + 1 > width:
                for k in range(i, j):
                    v_processed[k] = 0
            i = j;

    #generate peaks index
    peaksIndex, _ = ss.find_peaks(v_processed)
    #merge close peaks
    i = 0
    j = 1
    thresInterval = 70 # the smallest interval between two joints
    res = []
    while(j < len(peaksIndex)):
        if  j == len(peaksIndex) - 1:
            if v_processed[peaksIndex[i]] >= v_processed[peaksIndex[j]]:
                res.append(peaksIndex[i])
            else:
                res.append(peaksIndex[j])
            i = j
            j += 1       
        else:
            if peaksIndex[j] - peaksIndex[j-1] > thresInterval:
                j -= 1
                if v_processed[peaksIndex[i]] >= v_processed[peaksIndex[j]]:
                    res.append(peaksIndex[i])
                else:
                    res.append(peaksIndex[j])
                j += 1
                i = j
                j += 1       
            else:
                j += 1

    res = res[1:]

    trace0 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=v,
        name = 'compass',
        line = dict(
            color = ('rgb(25, 25, 25)'),
            width = 1)
    )

    trace1 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=v_processed,
        name = 'processed compass',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )

    trace2 = go.Scattergl(
        x = df.time[res] / timeConversionVal,
        y = v_processed[res],
        mode = 'markers',
        marker = dict(
            color = ('rgb(225, 0, 0)'),
            size = 5
        )
    )

    trace3 = go.Scattergl(
        x = df.time[peaksIndex] / timeConversionVal,
        y = v_processed[peaksIndex],
        mode = 'markers',
        marker = dict(
            color = ('rgb(0, 0, 225)'),
            size = 5
        )
    )

    data = [trace0, trace1, trace3, trace2]

    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Megnetic')
                      )

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html')
    
    return res

In [122]:
def jointDetectionByAcc(df, filename='jointDetectionByAcc'):
    x = df.accX
    y = df.accY
    z = df.accZ

    v = (df.accX ** 2 + df.accY ** 2 + df.accZ ** 2) ** (1/2)
    
    for i in range(len(v)): # - gravity from acc
        v[i] = v[i] - 98

    v_processed = abs(v)

    for i in range(len(v_processed)): #eliminate peak smaller than 20
        if v_processed[i] < 20:
            v_processed[i] = 0

    i = 0
    j = 0
    width = 25 #ger rid of peak whose width > 25 points
    while(j < len(v_processed)):
        if v_processed[i] == 0 and v_processed[j] == 0:
            i = i + 1
            j = j + 1
        elif v_processed[i] != 0 and v_processed[j] != 0:
            j = j + 1
        elif v_processed[i] != 0 and v_processed[j] == 0:
            if j - i + 1 > width:
                for k in range(i, j):
                    v_processed[k] = 0
            i = j;

    #generate peaks index
    peaksIndex, _ = ss.find_peaks(v_processed)
    #merge close peaks
    i = 0
    j = 1
    thresInterval = 70 # the smallest interval between two joints
    res = []
    while(j < len(peaksIndex)):
        if  j == len(peaksIndex) - 1:
            if v_processed[peaksIndex[i]] >= v_processed[peaksIndex[j]]:
                res.append(peaksIndex[i])
            else:
                res.append(peaksIndex[j])
            i = j
            j += 1       
        else:
            if peaksIndex[j] - peaksIndex[j-1] > thresInterval:
                j -= 1
                if v_processed[peaksIndex[i]] >= v_processed[peaksIndex[j]]:
                    res.append(peaksIndex[i])
                else:
                    res.append(peaksIndex[j])
                j += 1
                i = j
                j += 1       
            else:
                j += 1

    res = res[1:]

    trace0 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=v,
        name = 'acc',
        line = dict(
            color = ('rgb(25, 25, 25)'),
            width = 1)
    )

    trace1 = go.Scattergl(
        x=df.time/timeConversionVal,
        y=v_processed,
        name = 'processed acc',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )

    trace2 = go.Scattergl(
        x = df.time[res] / timeConversionVal,
        y = v_processed[res],
        mode = 'markers',
        marker = dict(
            color = ('rgb(225, 0, 0)'),
            size = 5
        )
    )

    trace3 = go.Scattergl(
        x = df.time[peaksIndex] / timeConversionVal,
        y = v_processed[peaksIndex],
        mode = 'markers',
        marker = dict(
            color = ('rgb(0, 0, 225)'),
            size = 5
        )
    )
    
#     traceAcc = go.Scattergl(
#         x = df.time / timeConversionVal,
#         y = (x ** 2 + y ** 2 + z ** 2) ** (1/2),
#         mode = 'lines',
#         line = dict(
#             color = ('rgb(0, 225, 0)')
#         )
#     )

    data = [trace0, trace1, trace3, trace2]#, traceAcc]

    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Acc(10 * m/s2)')
                      )

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html')
    
    return res

In [114]:
def speedCalculation(df, res, filename='speedCalculation'):
    time = velocityCal(df.time, res)

    #calculate mean of velocity
    errorPercent = 0.2
    mode = 6000

    con = []
    avg = []

    time = time.reset_index(drop=True)
    window = 5;

    avg = movingaverage(time, 10)
    
    
    for i in time.index:    
        try:
            # shift those points with double interval to avg, optional
            if i > 0 and i < len(time.index) and \
                time[i] < (2 + errorPercent) * time[i-1] and time[i] > (2 - errorPercent) * time[i-1] and\
                time[i] < (2 + errorPercent) * time[i+1] and time[i] > (2 - errorPercent) * time[i+1]:
                time[i] = time[i] #optional
#                 time[i] = time[i-1]
        except KeyError:
            time[i] = time[i]
        finally:   
            diff = abs(time[i] - avg[i])
            con.append(1 - diff/avg[i])

    #fill the result to df
    df['velocity'] = 0
    df['probability'] = 0
    df['avg'] = 0
    df['velocity'][res] = time
    df['probability'][res] = con
    df['avg'][res] = avg

    shrinkVal = 0.1 #other points between two joint, lower the possibility
    thresPro = 0.5 #threshold on probability for selecting a joint

    first = []
    #fill out empty velocity & probability in df
    #connect tow close confident points(probability > thresPro), calculate points between them by linear function
    #first & last segment, only use one point
    for index, row in df.iterrows():
        if len(first) == 0:
            if row['probability'] < thresPro:
#             if row['velocity'] > 1.5 * row['avg']:
                continue
            else:
                first = [row['velocity'], row['probability'], row['time'], index]
                for i in range(0, index):
                    df['velocity'][i] = first[0]
                    df['probability'][i] = first[1] - shrinkVal
        else:
            if index != len(df.index) -1:
                if row['probability'] < thresPro:
#                 if row['velocity'] > 1.5 * row['avg']:
                    continue
                else:
                    k = (row['velocity'] - first[0]) / (row['time'] - first[2])
                    b = row['velocity'] - k * row['time']
                    for i in range(first[3]+1, index):
                        avg2 = (df['avg'][index] + df['avg'][first[3]]) / 2
                        df['velocity'][i] = k * df['time'][i] + b
                        df['probability'][i] = (1 - abs(df['velocity'][i] - avg2) / avg2) - shrinkVal
                    first = [row['velocity'], row['probability'], row['time'], index]
            else:
                for i in range(first[3], len(df.index)):
                    df['velocity'][i] = first[0]
                    df['probability'][i] = first[1] - shrinkVal
    
#     ratio = 0.05
#     mvAvg = velocityFromAcc(df)
#     meanFront = abs(mean(mvAvg[0: int(ratio * len(df.index))]))
#     meanBack = abs(mean(mvAvg[int((1-ratio) * len(df.index)): len(df.index)]))
#     for index, row in df.iterrows():
#         if row['time'] < df.time.tail(1).tolist()[0] * ratio:
#             if abs(mvAvg[index]) < 15 and meanFront < 15:
#                 df['velocity'][index] = df['velocity'][int(ratio * len(df.index))]
#         if row['time'] > df.time.tail(1).tolist()[0] * (1-ratio):
#             if abs(mvAvg[index]) < 15 and meanBack < 15:
#                 df['velocity'][index] = df['velocity'][int((1-ratio) * len(df.index))]
            
    timeConversionVal = 60000

    traceAcceleration = go.Scattergl(
        x = df.time[res]/timeConversionVal,
        y = time,
        mode = 'markers',
        name = 'compass',
        marker = dict(
            color = ('rgb(0, 0, 255)')
        )
    )

    traceFit = go.Scattergl(
        x = df.time / timeConversionVal,
        y = df.velocity,
        mode = 'lines',
        name = 'velocity',
        line = dict(
            color = ('rgb(127, 127, 127)')
        )
    )


    data = [traceAcceleration, traceFit]

    layout = dict(title = filename,
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'timeInterval')
                  )
    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') 

In [121]:
def speedPlot(df, filename='speedPlot'):
    
    lenOfPipe = 40
    
    df['velocity'] = lenOfPipe / (df['velocity']/1000) #trans from time interval to velocity
    df['avg'] = lenOfPipe / (df['avg']/1000)
    
    traceV1 = go.Scattergl(
        x = df.time / timeConversionVal,
        y = df.velocity,
        mode = 'lines',
        name = 'Velocity',
        line = dict(
            color = ('rgb(0, 0, 255)')
        )
    )
    
    traceP1 = go.Scattergl(
        x = df.time / timeConversionVal,
        y = df.probability,
        mode = 'lines',
        name = 'Probability',
        line = dict(
            color = ('rgb(255, 0, 0)')
        )
    )
    
    data = [traceV1, traceP1]
    
    layout = dict(title = filename,
                      xaxis = dict(title = 'Time(minutes)'),
                      yaxis = dict(title = 'speed(feet/sec)'))
    
    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') 

In [123]:
run = 5
df = pd.read_csv('/Users/meng/Downloads/Wise_Run' + str(run) + '_Trimed.csv')
# res = jointDetectionByCompass(df, 'jointDetection' + str(run))
# speedCalculation(df, res, 'speedCalculation' + str(run))
# speedPlot(df, 'speedPlot' + str(run))

In [124]:
res = jointDetectionByAcc(df, 'jointDetection' + str(run))
speedCalculation(df, res, 'speedCalculation' + str(run))
speedPlot(df, 'speedPlot' + str(run))

In [127]:
df.to_csv('wise' + str(run) + ' with speed.csv')

In [13]:
from directionCalculation import getDirection, getDirectionByRotation, getTurningPoint

In [14]:
# run = '08'
# df = pd.read_csv('/Users/meng/Documents/dataPreprocessing/venv/source/sa_run' + run + '_trim.csv')
# df = pd.read_csv('/Users/meng/Documents/dataPreprocessing/venv/source/angle.csv')
angles = getDirection(df)
angles = butter_lowpass_filter(angles, 1, 50, 5).tolist() #lowpass
angles_rotation = getDirectionByRotation(df)
angles_rotation = butter_lowpass_filter(angles_rotation, 1, 50, 5).tolist() #lowpass

/Users/meng/Documents/dataPreprocessing/venv/lib/python3.6/directionCalculation.py:65: RuntimeWarning:

invalid value encountered in double_scalars

/Users/meng/Documents/dataPreprocessing/venv/lib/python3.6/directionCalculation.py:66: RuntimeWarning:

invalid value encountered in double_scalars



In [16]:
if len(angles) < len(df.index):
    length = len(df.index) - len(angles)
    for i in range(length):
        angles.append(0)
if len(angles_rotation) < len(df.index):
    length = len(df.index) - len(angles_rotation)
    for i in range(length):
        angles_rotation.append(0)
df['angles'] = 0
df['angles'] = angles
df['angles_rotation'] = 0
df['angles_rotation'] = angles_rotation

In [17]:
df.to_csv('run' + str(run) + 'with speedAngle.csv')

In [18]:
#get bend point
time, angle = getTurningPoint(df)
print(time)
print(angle)

[-44.870008904308456, -44.870008904308456, -8.364040732707245, -8.364040732707245, -3.8463917278206736, -3.8463917278206736, -45.30000455927362, -45.30000455927362, 16.509000004028394, 16.509000004028394, -84.74925011236896, -84.74925011236896, 76.64506791991829, 76.64506791991829]
[6.848333333333334, 6.8963, 7.001233333333333, 7.01835, 7.01955, 7.028416666666667, 7.198483333333333, 7.241783333333333, 8.84655, 8.970366666666667, 9.142183333333334, 9.198, 14.04255, 14.08895]
